In [1]:
from bertopic import BERTopic
from transformers import AutoTokenizer, AutoModel, pipeline
import pandas as pd
import pickle
import torch
import numpy as np
from bertopic.representation import KeyBERTInspired
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bertopic.vectorizers import ClassTfidfTransformer

/home/maaz-lfd/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-14 20:48:09.086455: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 20:48:21.122605: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-14 20:48:21.128207: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plu

In [12]:
df = pd.read_csv(
    "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5_CT_metdata.csv"
)
df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,5yhe786e,3bb07ea10432f7738413dff9816809cc90f03f99,PMC,Debate: Transfusing to normal haemoglobin leve...,10.1186/cc987,PMC137267,11299062.0,no-cc,Recent evidence suggests that critically ill p...,2001-03-08,"Alvarez, Gonzalo; Hébert, Paul C; Szick, Sharyn",Crit Care,NaN,NaN,NaN,document_parses/pdf_json/3bb07ea10432f7738413d...,document_parses/pmc_json/PMC137267.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
1,8zchiykl,5806726a24dc91de3954001effbdffd7a82d54e2,PMC,The 21st International Symposium on Intensive ...,10.1186/cc1013,PMC137274,11353930.0,no-cc,The 21st International Symposium on Intensive ...,2001-05-02,"Ball, Jonathan; Venn, Richard",Crit Care,NaN,NaN,NaN,document_parses/pdf_json/5806726a24dc91de39540...,document_parses/pmc_json/PMC137274.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
2,x7wg7e9s,e7f311d3c752063f75f28de85abd5f82eacab413,PMC,Epicatechins Purified from Green Tea (Camellia...,10.1093/ecam/nel003,PMC1475929,16786054.0,no-cc,The anticancer potential of catechins derived ...,2006-04-25,"Ravindranath, Mepur H.; Saravanan, Thiruverkad...",Evid Based Complement Alternat Med,NaN,NaN,NaN,document_parses/pdf_json/e7f311d3c752063f75f28...,document_parses/pmc_json/PMC1475929.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
3,utglk4af,babad3896fca8ed7c89d9b785c08e7c42a48a6a2,PMC,Markers of exacerbation severity in chronic ob...,10.1186/1465-9921-7-74,PMC1481583,16686949.0,cc-by,BACKGROUND: Patients with chronic obstructive ...,2006-05-10,"Franciosi, Luigi G; Page, Clive P; Celli, Bart...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/babad3896fca8ed7c89d9...,document_parses/pmc_json/PMC1481583.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
4,hwlvk68z,0dcf7336a3569d4103dc5be47ddd6b583dea98f0,PMC,Transmission patterns of smallpox: systematic ...,10.1186/1471-2458-6-126,PMC1525174,16677388.0,cc-by,BACKGROUND: Because smallpox (variola major) m...,2006-05-05,"Bhatnagar, Vibha; Stoto, Michael A; Morton, Sa...",BMC Public Health,NaN,NaN,NaN,document_parses/pdf_json/0dcf7336a3569d4103dc5...,document_parses/pmc_json/PMC1525174.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7797,9th2jiq6,97229e8af52b75aa9ac0f0eb530bf89f7f5ec134,Medline; PMC,Cyclophilin A Associates with Enterovirus-71 V...,10.1371/journal.ppat.1004422,PMC4183573,25275585.0,cc-by,Viruses utilize host factors for their efficie...,2014-10-02,"Qing, Jie; Wang, Yaxin; Sun, Yuna; Huang, Jiao...",PLoS Pathog,NaN,NaN,NaN,document_parses/pdf_json/97229e8af52b75aa9ac0f...,document_parses/pmc_json/PMC4183573.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/25275585/;...,9113317.0
7798,bpzwx27u,be040de2385e16c635984fb0c4b5be9be8707301,Elsevier; Medline; PMC,Cardiovascular Considerations for the Internis...,10.1016/j.amjmed.2020.06.005,PMC7347494,32653420.0,no-cc,It is clear that existing cardiovascular disea...,2020-07-10,"Massoomi, Michael R.; Anderson, R. David; Ahme...",Am J Med,NaN,NaN,NaN,document_parses/pdf_json/be040de2385e16c635984...,document_parses/pmc_json/PMC7347494.xml.json,https://www.sciencedirect.com/science/article/...,220435265.0
7799,1f47gvys,d78837afeedbb3c9dfa91a675aa03d34ef5a4387,Medline; PMC,The role of airway macrophages in apoptotic ce...,10.1007/s00281-016-0555-3,PMC4896990,26957481.0,cc-by,Acute and chronic inflammatory responses in th...,2016-03-08,"Grabiec, Aleksander M.; Hussell, Tracy",Semin Immunopathol,NaN,NaN,NaN,document_parses/pdf_json/d78837afeedbb3c9dfa91...,document_parses/pmc_json/PMC4896990.xml.json,https://doi.org/10.1007/s00281-016-0555-3; htt...,804810.0
7800,sqy03dks,6556edfa043bccf843e9ae077ff8005106764b7e,Elsevier; Medline; PMC,N-Acetylcysteine: a potential therapeutic agen...,10.1016/j.mehy.2020.109862,P

#### Remove rows that don't have any author

In [13]:
df2 = pd.read_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/GS_paper_metadata.csv')
df2= df2[df2['author'].notna()]
df2

,pmcid,doi,citations,author,author_link,author_h_index,publisher_link,author_citations
0,PMC137267,10.1186/cc987,42,PC Hébert,https://scholar.google.com//citations?user=EWs...,93,https://link.springer.com/article/10.1186/cc987,41828
1,PMC137274,10.1186/cc1013,5,J Ball,https://scholar.google.com//citations?user=Laq...,30,https://ccforum.biomedcentral.com/articles/10....,8396
2,PMC1475929,10.1093/ecam/nel003,0,MH Ravindranath;TS Saravanan,https://scholar.google.com//citations?user=K26...,9,https://www.hindawi.com/journals/ecam/2006/105...,355
3,PMC1481583,10.1186/1465-9921-7-74,106,BR Celli;MJ Walker;M Danhof;KF Rabe,https://scholar.google.com//citations?user=TVa...,128,https://link.springer.com/article/10.1186/1465...,81730
4,PMC1525174,10.1186/1471-2458-6-126,0,MA Stoto;SC Morton;R Boer,https://scholar.google.com//citations?user=_Dc...,52,https://link.springer.com/article/10.1186/1471...,9883
...,...,...,...,...,...,...,...,...
7790,PMC5722503,10.18632/oncotarget.22106,8,MS Lindström;J Bartek,https://scholar.google.com//citations?user=on5...,150,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,113358
7793,PMC7339790,10.1038/s41577-020-0389-z,382,A Sette;S Crotty,https://scholar.google.com//citations?user=Ep4...,99,https://www.nature.com/articles/s41577-020-0389-z,49506
7794,PMC7108131,10.1093/cid/ciz760,5,AM Rojek;PW Horby,https://scholar.google.com//citations?user=M4P...,87,https://academic.oup.com/cid/article-abstract/...,66986
7797,PMC4183573,10.1371/journal.ppat.1004422,84,D Su,https://scholar.google.com//citations?user=4mA...,23,https://journals.plos.org/plospathogens/articl...,2407


In [14]:
df = df[df['pmcid'].isin(df2['pmcid'].to_list())]
df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,5yhe786e,3bb07ea10432f7738413dff9816809cc90f03f99,PMC,Debate: Transfusing to normal haemoglobin leve...,10.1186/cc987,PMC137267,11299062.0,no-cc,Recent evidence suggests that critically ill p...,2001-03-08,"Alvarez, Gonzalo; Hébert, Paul C; Szick, Sharyn",Crit Care,NaN,NaN,NaN,document_parses/pdf_json/3bb07ea10432f7738413d...,document_parses/pmc_json/PMC137267.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
1,8zchiykl,5806726a24dc91de3954001effbdffd7a82d54e2,PMC,The 21st International Symposium on Intensive ...,10.1186/cc1013,PMC137274,11353930.0,no-cc,The 21st International Symposium on Intensive ...,2001-05-02,"Ball, Jonathan; Venn, Richard",Crit Care,NaN,NaN,NaN,document_parses/pdf_json/5806726a24dc91de39540...,document_parses/pmc_json/PMC137274.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
2,x7wg7e9s,e7f311d3c752063f75f28de85abd5f82eacab413,PMC,Epicatechins Purified from Green Tea (Camellia...,10.1093/ecam/nel003,PMC1475929,16786054.0,no-cc,The anticancer potential of catechins derived ...,2006-04-25,"Ravindranath, Mepur H.; Saravanan, Thiruverkad...",Evid Based Complement Alternat Med,NaN,NaN,NaN,document_parses/pdf_json/e7f311d3c752063f75f28...,document_parses/pmc_json/PMC1475929.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
3,utglk4af,babad3896fca8ed7c89d9b785c08e7c42a48a6a2,PMC,Markers of exacerbation severity in chronic ob...,10.1186/1465-9921-7-74,PMC1481583,16686949.0,cc-by,BACKGROUND: Patients with chronic obstructive ...,2006-05-10,"Franciosi, Luigi G; Page, Clive P; Celli, Bart...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/babad3896fca8ed7c89d9...,document_parses/pmc_json/PMC1481583.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
4,hwlvk68z,0dcf7336a3569d4103dc5be47ddd6b583dea98f0,PMC,Transmission patterns of smallpox: systematic ...,10.1186/1471-2458-6-126,PMC1525174,16677388.0,cc-by,BACKGROUND: Because smallpox (variola major) m...,2006-05-05,"Bhatnagar, Vibha; Stoto, Michael A; Morton, Sa...",BMC Public Health,NaN,NaN,NaN,document_parses/pdf_json/0dcf7336a3569d4103dc5...,document_parses/pmc_json/PMC1525174.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7790,i1il6jjs,5a5a25d733b227ba661f3ba8f69af5886999f02c,Medline; PMC,Human cytomegalovirus and Herpes Simplex type ...,10.18632/oncotarget.22106,PMC5722503,29228551.0,cc-by,Human cytomegalovirus (HCMV) utilizes RNA poly...,2017-10-29,"Kostopoulou, Ourania N.; Wilhelmi, Vanessa; Ra...",Oncotarget,NaN,NaN,NaN,document_parses/pdf_json/5a5a25d733b227ba661f3...,document_parses/pmc_json/PMC5722503.xml.json,https://doi.org/10.18632/oncotarget.22106; htt...,30028157.0
7793,bw6a5gmy,e8c436a170cb0a1b7024effb168bb8a7214f2021,Medline; PMC,Pre-existing immunity to SARS-CoV-2: the known...,10.1038/s41577-020-0389-z,PMC7339790,32636479.0,no-cc,T cell reactivity against SARS-CoV-2 was obser...,2020-07-07,"Sette, Alessandro; Crotty, Shane",Nat Rev Immunol,NaN,NaN,NaN,document_parses/pdf_json/e8c436a170cb0a1b7024e...,document_parses/pmc_json/PMC7339790.xml.json,https://doi.org/10.1038/s41577-020-0389-z; htt...,220381745.0
7794,rpwccvqi,96abc8769d5840bdeac8590d795561706f34b4c4,Medline; PMC,Core Minimal Datasets to Advance Clinical Rese...,10.1093/cid/ciz760,PMC7108131,31406989.0,cc-by,The Ebola virus disease outbreak in west Afric...,2020-02-15,"Rojek, Amanda M; Moran, James; Horby, Peter W",Clin Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/96abc8769d5840bdeac85...,document_parses/pmc_json/PMC7108131.xml.json,https://doi.org/10.1093/cid/ciz760; https://ww...,199549631.0
7797,9th2jiq6,97229e8af52b75aa9ac0f0eb530bf89f7f5ec134,Medline; PMC,Cyclophilin A Associates with Enterovirus-71 V...,10.1371/journal.ppat.1004422,PMC4183573,25275585.0,cc-by,Viruses

In [15]:
df = df.reset_index()

In [17]:
df.to_csv("/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5_CT_metdata.csv",index=False)

In [4]:
df2.isnull().any()

pmcid               False
doi                 False
citations           False
author              False
author_link         False
author_h_index      False
publisher_link      False
author_citations    False
dtype: bool

In [5]:
pmcid = df2['pmcid'].tolist()
pmcid

['PMC137267',
 'PMC137274',
 'PMC1475929',
 'PMC1481583',
 'PMC1525174',
 'PMC1540438',
 'PMC1764036',
 'PMC2206432',
 'PMC2206439',
 'PMC2569416',
 'PMC2707627',
 'PMC2758170',
 'PMC2811908',
 'PMC2825274',
 'PMC2846346',
 'PMC2851514',
 'PMC2892358',
 'PMC2909231',
 'PMC2921955',
 'PMC2923618',
 'PMC2949385',
 'PMC2950129',
 'PMC2970549',
 'PMC2970802',
 'PMC2990691',
 'PMC3001050',
 'PMC3002947',
 'PMC3003998',
 'PMC3010100',
 'PMC3022902',
 'PMC3030543',
 'PMC3037927',
 'PMC3060167',
 'PMC3061093',
 'PMC3064590',
 'PMC3065411',
 'PMC3068961',
 'PMC3070035',
 'PMC3104979',
 'PMC3137391',
 'PMC3159992',
 'PMC3161824',
 'PMC3163371',
 'PMC3173582',
 'PMC3207872',
 'PMC3218971',
 'PMC3220036',
 'PMC3229615',
 'PMC3235500',
 'PMC3239386',
 'PMC3260104',
 'PMC3260331',
 'PMC3268102',
 'PMC3270559',
 'PMC3274434',
 'PMC3292808',
 'PMC3311214',
 'PMC3342007',
 'PMC3344869',
 'PMC3372491',
 'PMC3387643',
 'PMC3410888',
 'PMC3433438',
 'PMC3439243',
 'PMC3441198',
 'PMC3472818',
 'PMC3486884

In [6]:
df = df[df['pmcid'].isin(pmcid)]
df = df.reset_index()
df

,pmcid,abstract
0,PMC137267,Recent evidence suggests that critically ill p...
1,PMC137274,The 21st International Symposium on Intensive ...
2,PMC1475929,The anticancer potential of catechins derived ...
3,PMC1481583,BACKGROUND: Patients with chronic obstructive ...
4,PMC1525174,BACKGROUND: Because smallpox (variola major) m...
...,...,...
7810,PMC5722503,Human cytomegalovirus (HCMV) utilizes RNA poly...
7813,PMC7339790,T cell reactivity against SARS-CoV-2 was obser...
7814,PMC7108131,The Ebola virus disease outbreak in west Afric...
7817,PMC4183573,Viruses utilize host factors for their efficie...


#### ....

Load **PubMedELECTRA** model

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/BiomedNLP-PubMedELECTRA-large-uncased-abstract"  
)
model = AutoModel.from_pretrained(
    "microsoft/BiomedNLP-PubMedELECTRA-large-uncased-abstract"
)
# model = pipeline(model= "microsoft/BiomedNLP-PubMedELECTRA-large-uncased-abstract")

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedELECTRA-large-uncased-abstract were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load **ClinicalBert** Model

In [29]:
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

In [8]:
torch.cuda.is_available()

True

In [9]:
# Set CUDA device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda")
device

device(type='cuda')

In [30]:
model.to(device)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(

In [31]:
model.eval()
first_embedding = True
for i in range(len(df)):
    print("count ", i + 1)
    tokens = tokenizer(
        df["abstract"][i],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokens = {key: val.to(device) for key, val in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
        embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        if first_embedding == True:
            with open(
                "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/slice_abstract_embedding_clinicalBERT",
                "wb",
            ) as f:
                pickle.dump(embedding, f)

            first_embedding = False
        else:
            saved_emb = pickle.loads(
                open(
                    "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/slice_abstract_embedding_clinicalBERT",
                    "rb",
                ).read()
            )
            concat_emb = np.concatenate((embedding, saved_emb))
            print("shape after concatination ", concat_emb.shape)
            with open(
                "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/slice_abstract_embedding_clinicalBERT",
                "wb",
            ) as f:
                pickle.dump(concat_emb, f)

count  1
count  2
shape after concatination  (2, 768)
count  3
shape after concatination  (3, 768)
count  4
shape after concatination  (4, 768)
count  5
shape after concatination  (5, 768)
count  6
shape after concatination  (6, 768)
count  7
shape after concatination  (7, 768)
count  8
shape after concatination  (8, 768)
count  9
shape after concatination  (9, 768)
count  10
shape after concatination  (10, 768)
count  11
shape after concatination  (11, 768)
count  12
shape after concatination  (12, 768)
count  13
shape after concatination  (13, 768)
count  14
shape after concatination  (14, 768)
count  15
shape after concatination  (15, 768)
count  16
shape after concatination  (16, 768)
count  17
shape after concatination  (17, 768)
count  18
shape after concatination  (18, 768)
count  19
shape after concatination  (19, 768)
count  20
shape after concatination  (20, 768)
count  21
shape after concatination  (21, 768)
count  22
shape after concatination  (22, 768)
count  23
shape afte

In [32]:
concat_emb.shape

(3632, 768)

In [106]:
representation_model = KeyBERTInspired()

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=50, n_components=200, min_dist=0.0, metric="cosine", random_state=42)

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=10, metric="euclidean", cluster_selection_method="eom", prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))
# vectorizer_model = TfidfVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with 
# a `bertopic.representation` model
representation_model = KeyBERTInspired()

topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model,  ctfidf_model=ctfidf_model 
                       ,top_n_words=10, verbose=True)

#### TOPIC MODELING USING CLINICAL BERT

In [71]:
clinical_bert_embeddings = pickle.loads(  open("/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/slice_abstract_embedding_clinicalBERT","rb",).read() )

In [107]:
topics, probs = topic_model.fit_transform(df["abstract"].to_list(), clinical_bert_embeddings)

2023-10-10 16:48:58,429 - BERTopic - Reduced dimensionality
2023-10-10 16:49:04,860 - BERTopic - Clustered reduced embeddings


In [108]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2067,-1_patients_covid19_disease_clinical,"[patients, covid19, disease, clinical, respira...",[Therapeutic options for coronavirus disease 2...
1,0,397,0_patients_clinical_covid19_disease,"[patients, clinical, covid19, disease, infecti...",[BACKGROUND: The 2019 novel coronavirus diseas...
2,1,316,1_covid19_patients_clinical_virus,"[covid19, patients, clinical, virus, respirato...",[Existing and emerging viral CNS infections ar...
3,2,122,2_covid19_clinical_disease_viruses,"[covid19, clinical, disease, viruses, patients...",[BACKGROUND: Clinical trials applying iota-car...
4,3,102,3_treatment_patients_virus_disease,"[treatment, patients, virus, disease, studies,...",[BACKGROUND: Influenza and pneumococcal vaccin...
5,4,94,4_virus_cells_clinical_vaccines,"[virus, cells, clinical, vaccines, cell, vacci...",[Hepatitis C Virus (HCV) infects 2% of the wor...
6,5,81,5_patients_covid19_development_disease,"[patients, covid19, development, disease, vacc...",[INTRODUCTION: Community-acquired pneumonia (C...
7,6,74,6_patients_infection_treatment_respiratory,"[patients, infection, treatment, respiratory, ...",[Pulmonary diseases due to mycobacteria cause ...
8,7,45,7_dpp4_calves_cpc_patients,"[dpp4, calves, cpc, patients, isolation, vacci...",[Chronic obstructive pulmonary disease (COPD) ...
9,8,38,8_surfactant_virus_infectious_clinical,"[surfactant, virus, infectious, clinical, repl...",[The global spread of the severe acute respira...


In [109]:
topic_model.get_topic_info()['Representation'][1]

['patients',
 'clinical',
 'covid19',
 'disease',
 'infection',
 'respiratory',
 'treatment',
 'viral',
 'health',
 'virus']

In [110]:
# Intertopic Distance Map
topic_model.visualize_topics(title='<b>CLINICAL BERT</b>')

In [111]:
# Hierarchical Clustering
topic_model.visualize_hierarchy(title="<b>CLINICAL BERT</b>")

In [112]:
abs_topics = pd.DataFrame({"topic": topics, "pmcid":df['pmcid'], "abstracts": df["abstract"].to_list()})

In [113]:
abs_topics.to_excel(
    "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/topics/topic_abstract_map.xlsx",
    index=False,
)

In [104]:
topic_model.save(
    "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/topics/topic_model",
    serialization="pickle",
)

#### TOPIC MODELING USING PUBMED ELECTRA

In [76]:
pubmed_electra_embeddings = pickle.loads(  open("/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/slice_abstract_embedding_PubMedELECTRA","rb",).read() )

In [114]:
topics, probs = topic_model.fit_transform(df["abstract"].to_list(), pubmed_electra_embeddings)

2023-10-10 17:05:43,617 - BERTopic - Reduced dimensionality
2023-10-10 17:05:52,497 - BERTopic - Clustered reduced embeddings


In [115]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1943,-1_patients_covid19_clinical_disease,"[patients, covid19, clinical, disease, respira...",[Severe influenza infections are often associa...
1,0,263,0_patients_covid19_infection_clinical,"[patients, covid19, infection, clinical, disea...",[Acute respiratory tract infections are a majo...
2,1,244,1_patients_clinical_covid19_respiratory,"[patients, clinical, covid19, respiratory, dis...",[The outbreak of coronavirus disease 2019 (COV...
3,2,88,2_patients_influenza_ards_used,"[patients, influenza, ards, used, clinical, re...",[Please cite this paper as: Howard et al. (201...
4,3,75,3_patients_covid19_respiratory_disease,"[patients, covid19, respiratory, disease, biof...",[BACKGROUND: We combined hospital-based survei...
5,4,72,4_patients_disease_cells_data,"[patients, disease, cells, data, clinical, rev...",[BACKGROUND: Patients with influenza complicat...
6,5,71,5_patients_covid19_care_median,"[patients, covid19, care, median, dvt, rsv, di...",[The world has developed an elaborate global h...
7,6,64,6_virus_covid19_viral_trials,"[virus, covid19, viral, trials, respiratory, t...",[BACKGROUND: Mortality of patients with corona...
8,7,55,7_covid19_clinical_health_virus,"[covid19, clinical, health, virus, cd47, resea...",[BACKGROUND: The coronavirus disease (COVID-19...
9,8,54,8_cells_patients_2c_viral,"[cells, patients, 2c, viral, 1b, cell, liver i...",[Background: Liver injury commonly occurs in p...


In [124]:
topic_df = topic_model.get_topic_info()
topic_df[['Topic','Count']]

,Topic,Count
0,-1,1943
1,0,263
2,1,244
3,2,88
4,3,75
5,4,72
6,5,71
7,6,64
8,7,55
9,8,54


In [116]:
# Intertopic Distance Map
topic_model.visualize_topics(title='<b>PUBMED ELECTRA</b>')

In [117]:
# Hierarchical Clustering
topic_model.visualize_hierarchy(title="<b>PUBMED ELECTRA</b>")

In [118]:
abs_topics = pd.DataFrame({"topic": topics, "pmcid":df['pmcid'], "abstracts": df["abstract"].to_list()})

In [119]:
abs_topics.to_excel(
    "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/topics/topic_abstract_map_2.xlsx",
    index=False,
)